## Import paczek potrzebnych do laboratorium

In [534]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error
# Obliczanie przedziału ufności
from sklearn.metrics import mean_squared_error

## Liczba uzytkownikow w kolejnych kwartalach (Facebook)

In [535]:
user_count_q = {
    'Q':
        ['Q3 08', 'Q1 09', 'Q2 09', 'Q3 09', 'Q4 09', 'Q1 10',
         'Q2 10', 'Q3 10', 'Q4 10', 'Q1 11', 'Q2 11', 'Q3 11',
         'Q4 11', 'Q1 12', 'Q2 12', 'Q3 12', 'Q4 12', 'Q1 13',
         'Q2 13', 'Q3 13', 'Q4 13', 'Q1 14', 'Q2 14', 'Q3 14',
         'Q4 14', 'Q1 15', 'Q2 15', 'Q3 15', 'Q4 15', 'Q1 16',
         'Q2 16', 'Q3 16', 'Q4 16', 'Q1 17', 'Q2 17', 'Q3 17', 'Q4 17'],
    'User count':
        [100, 197, 242, 305, 360, 431, 482, 550, 608, 680, 739, 800,
         845, 901, 955, 1007, 1056, 1110, 1155, 1189, 1228, 1276, 1317,
         1350, 1393, 1441, 1490, 1545, 1591, 1654, 1712, 1788, 1860, 1936,
         2006, 2072, 2129]
}

## Liczba uzytkownikow w kolejnych kwartalach (Facebook) - dane z lat 2008-2017 - usuwanie roku 2007 gdyz nie ma danych dla poprzedniej tabeli z tego roku


In [536]:
fb_stats_y_old = {
    'Y': [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    'Income': [153, 272, 777, 1974, 3711, 5089, 7872, 12466, 17928, 27638, 40653],
    'Profit': [-138, -56, 229, 606, 1000, 53, 1500, 2940, 3688, 10217, 15934],
    'Employment': [450, 850, 1218, 2127, 3200, 4619, 6337, 9199, 12691, 17048, 25105]
}

# Poprawione dane z 2008-2017
fb_stats_y = {
    'Y': [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    'Income': [272, 777, 1974, 3711, 5089, 7872, 12466, 17928, 27638, 40653],
    'Profit': [-56, 229, 606, 1000, 53, 1500, 2940, 3688, 10217, 15934],
    'Employment': [850, 1218, 2127, 3200, 4619, 6337, 9199, 12691, 17048, 25105]
}

## Tworzenie DataFrame z danymi 

In [537]:
user_count_df = pd.DataFrame(data=user_count_q)
fb_stats_y_df = pd.DataFrame(data=fb_stats_y)
print(fb_stats_y_df)
print(user_count_df)

      Y  Income  Profit  Employment
0  2008     272     -56         850
1  2009     777     229        1218
2  2010    1974     606        2127
3  2011    3711    1000        3200
4  2012    5089      53        4619
5  2013    7872    1500        6337
6  2014   12466    2940        9199
7  2015   17928    3688       12691
8  2016   27638   10217       17048
9  2017   40653   15934       25105
        Q  User count
0   Q3 08         100
1   Q1 09         197
2   Q2 09         242
3   Q3 09         305
4   Q4 09         360
5   Q1 10         431
6   Q2 10         482
7   Q3 10         550
8   Q4 10         608
9   Q1 11         680
10  Q2 11         739
11  Q3 11         800
12  Q4 11         845
13  Q1 12         901
14  Q2 12         955
15  Q3 12        1007
16  Q4 12        1056
17  Q1 13        1110
18  Q2 13        1155
19  Q3 13        1189
20  Q4 13        1228
21  Q1 14        1276
22  Q2 14        1317
23  Q3 14        1350
24  Q4 14        1393
25  Q1 15        1441
26  Q2 15 

In [538]:
def convert_quarter_to_year(quarter) -> int:
    year_str = quarter.split(' ')[1]
    year_int = 2000 + int(year_str)
    return year_int


In [539]:
user_count_y_df = user_count_df.copy()
user_count_y_df['Q'] = user_count_df['Q'].map(convert_quarter_to_year)
# Alternatywa byloby zaladowanie danych z Q4 jako max tez dobrze
user_count_y_df = user_count_y_df
user_count_y_df = user_count_y_df.groupby('Q', as_index=False).max()
user_count_y_df.rename(columns={'Q': 'Y'}, inplace=True)
print(user_count_y_df)

      Y  User count
0  2008         100
1  2009         360
2  2010         608
3  2011         845
4  2012        1056
5  2013        1228
6  2014        1393
7  2015        1591
8  2016        1860
9  2017        2129


In [540]:
fb_stats_y_df = pd.DataFrame(data=fb_stats_y)
# Calculate costs
fb_stats_y_df['Costs'] = fb_stats_y_df['Income'] - fb_stats_y_df['Profit']
# fb_stats_y_df.drop([0], inplace=True)
fb_stats_y_df.reset_index(drop=True, inplace=True)
fb_stats_y_df.insert(3, 'User count', user_count_y_df['User count'])
print(fb_stats_y_df)


      Y  Income  Profit  User count  Employment  Costs
0  2008     272     -56         100         850    328
1  2009     777     229         360        1218    548
2  2010    1974     606         608        2127   1368
3  2011    3711    1000         845        3200   2711
4  2012    5089      53        1056        4619   5036
5  2013    7872    1500        1228        6337   6372
6  2014   12466    2940        1393        9199   9526
7  2015   17928    3688        1591       12691  14240
8  2016   27638   10217        1860       17048  17421
9  2017   40653   15934        2129       25105  24719


## Tworzenie wykresu iloczynu kartezjanskiego w celu wyzanczeniach zaleznosci miedzy danymi

In [541]:
pd.plotting.scatter_matrix(fb_stats_y_df, figsize=(10, 10))

<IPython.core.display.Javascript object>

array([[<Axes: xlabel='Y', ylabel='Y'>,
        <Axes: xlabel='Income', ylabel='Y'>,
        <Axes: xlabel='Profit', ylabel='Y'>,
        <Axes: xlabel='User count', ylabel='Y'>,
        <Axes: xlabel='Employment', ylabel='Y'>,
        <Axes: xlabel='Costs', ylabel='Y'>],
       [<Axes: xlabel='Y', ylabel='Income'>,
        <Axes: xlabel='Income', ylabel='Income'>,
        <Axes: xlabel='Profit', ylabel='Income'>,
        <Axes: xlabel='User count', ylabel='Income'>,
        <Axes: xlabel='Employment', ylabel='Income'>,
        <Axes: xlabel='Costs', ylabel='Income'>],
       [<Axes: xlabel='Y', ylabel='Profit'>,
        <Axes: xlabel='Income', ylabel='Profit'>,
        <Axes: xlabel='Profit', ylabel='Profit'>,
        <Axes: xlabel='User count', ylabel='Profit'>,
        <Axes: xlabel='Employment', ylabel='Profit'>,
        <Axes: xlabel='Costs', ylabel='Profit'>],
       [<Axes: xlabel='Y', ylabel='User count'>,
        <Axes: xlabel='Income', ylabel='User count'>,
        <Axes: xla

## Dane dla przyszlych lat (2018-2022) potrzbne do weryfikowania predyckji modelu

In [542]:
user_count_q_test = {
    'Q':
        ['Q1 18', 'Q2 18', 'Q3 18', 'Q4 18',
         'Q1 19', 'Q2 19', 'Q3 19', 'Q4 19',
         'Q1 20', 'Q2 20', 'Q3 20', 'Q4 20',
         'Q1 21', 'Q2 21', 'Q3 21', 'Q4 21',
         'Q1 22', 'Q2 22', 'Q3 22', 'Q4 22'],
    'User count':
        [2196, 2234, 2271, 2320, 2375, 2414, 2449, 2498, 2603, 2701, 2740, 2797,
         2853, 2895, 2910, 2912, 2936, 2934, 2958, 2963]
}

fb_stats_y_test = {
    'Y': [2018, 2019, 2020, 2021, 2022],
    'Income': [55838, 70697, 85965, 117929, 116609],
    'Profit': [22112, 18485, 29146, 39370, 23200],
    'Costs': [33726, 52212, 56819, 78559, 93409],
    'Employment': [35587, 44942, 58604, 71970, 87314],
    'User count': [2320, 2498, 2797, 2912, 2963]
}

In [543]:
user_count_test_df = pd.DataFrame(data=user_count_q_test)
fb_stats_y_test_df = pd.DataFrame(data=fb_stats_y_test)
print(user_count_test_df)
print(fb_stats_y_test_df)

        Q  User count
0   Q1 18        2196
1   Q2 18        2234
2   Q3 18        2271
3   Q4 18        2320
4   Q1 19        2375
5   Q2 19        2414
6   Q3 19        2449
7   Q4 19        2498
8   Q1 20        2603
9   Q2 20        2701
10  Q3 20        2740
11  Q4 20        2797
12  Q1 21        2853
13  Q2 21        2895
14  Q3 21        2910
15  Q4 21        2912
16  Q1 22        2936
17  Q2 22        2934
18  Q3 22        2958
19  Q4 22        2963
      Y  Income  Profit  Costs  Employment  User count
0  2018   55838   22112  33726       35587        2320
1  2019   70697   18485  52212       44942        2498
2  2020   85965   29146  56819       58604        2797
3  2021  117929   39370  78559       71970        2912
4  2022  116609   23200  93409       87314        2963


## Funkcja do obliczania statystyki modelu

In [544]:
def calculate_model_stats(y_hat, y_true, X) -> dict:
    # Dodajemy kolumne jedynek dla wyrazu wolnego 
    X = np.insert(X, 0, 1, axis=1)

    residuals = y_true - y_hat
    # Obliczamy sume kwadratow reszt(RSS - Residual Sum of Squares)
    # RSS = e^T * e, gdzie e to wektor reszt
    residuals_sum_squared = residuals.T @ residuals

    # Obliczamy estymator wariancji resztowej (σ²)
    # σ² = RSS/(n-k), gdzie:
    # n = liczba obserwacji (y_hat.shape[0])
    # k = liczba parametrów modelu (X.shape[1])
    standard_variance = residuals_sum_squared[0, 0] / (y_hat.shape[0] - X.shape[1])

    # Var(β̂) = σ² × (X^T × X)^(-1)
    model_coefficients_covariance = standard_variance * np.linalg.inv(X.T @ X)  # Liczymy macierz kowariancji wspl. modelu
    stats = dict()

    # Obliczamy błędy standardowe dla każdego współczynnika
    # SE(β̂ᵢ) = √Var(β̂ᵢ)
    for number in range(model_coefficients_covariance.shape[0]):
        stats['Standard errror a' + str(number)] = np.sqrt(model_coefficients_covariance[number, number])

    # Na koniec blad standardowy modelu
    stats['Standard error e'] = np.sqrt(standard_variance)
    return stats



## Funkcje pomocnicze dla oceny predykcji modelu MSE, RMSE MAE i MAPE

In [545]:
def calculate_prediction_metrics(y_true, y_pred):
    # Parametry funkcji: y_true - rzeczywiste wartości, y_pred - przewidywane wartości
    # Konwersja do numpy array dla pewności
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # Mean Squared Error (MSE)
    mse = np.mean((y_true - y_pred) ** 2)
    
    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    
    # Mean Absolute Error (MAE)
    mae = np.mean(np.abs(y_true - y_pred))
    
    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # R-squared (R²)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2
    }

def print_metrics(metrics, model_name="Model"):
    def get_error_rating(metric_name, value):
        if metric_name == 'MAPE':
            if value < 10: return "Bardzo dobry"
            elif value < 20: return "Dobry"
            elif value < 30: return "Umiarkowany"
            else: return "Słaby"
        elif metric_name == 'R2':
            if value > 0.9: return "Bardzo dobry"
            elif value > 0.7: return "Dobry"
            elif value > 0.5: return "Umiarkowany"
            else: return "Słaby"
        return ""  # dla pozostałych metryk nie wyświetlamy oceny

    def get_unit(metric_name):
        units = {
            'MSE': '(jednostka²)',
            'RMSE': '(jednostka)',
            'MAE': '(jednostka)',
            'MAPE': '%',
            'R2': '(0-1)'
        }
        return units.get(metric_name, '')

    print(f"\nMetryki oceny dla {model_name}:")
    print("-" * 60)
    print(f"{'Metryka':<10} {'Wartość':<15} {'Jednostka':<15} {'Ocena':<15}")
    print("-" * 60)
    
    for metric, value in metrics.items():
        unit = get_unit(metric)
        rating = get_error_rating(metric, value)
        print(f"{metric:<10} {value:>15.4f} {unit:<15} {rating:<15}")
        
        
def print_metrics_with_specific_units(metrics, model_name="Model", unit_name=""):
    def get_error_rating(metric_name, value):
        if metric_name == 'MAPE':
            if value < 10: return "Bardzo dobry"
            elif value < 20: return "Dobry"
            elif value < 30: return "Umiarkowany"
            else: return "Słaby"
        elif metric_name == 'R2':
            if value > 0.9: return "Bardzo dobry"
            elif value > 0.7: return "Dobry"
            elif value > 0.5: return "Umiarkowany"
            else: return "Słaby"
        return ""

    def get_unit(metric_name, unit_name):
        units = {
            'MSE': f"({unit_name}²)",
            'RMSE': f"({unit_name})",
            'MAE': f"({unit_name})",
            'MAPE': '%',
            'R2': '(0-1)'
        }
        return units.get(metric_name, '')

    print(f"\nMetryki oceny dla {model_name}:")
    print("-" * 70)
    print(f"{'Metryka':<10} {'Wartość':<15} {'Jednostka':<20} {'Ocena':<15}")
    print("-" * 70)
    
    for metric, value in metrics.items():
        unit = get_unit(metric, unit_name)
        rating = get_error_rating(metric, value)
        print(f"{metric:<10} {value:>15.4f} {unit:<20} {rating:<15}")

## Funkcje pomocnicze dla szeregow czasowych z uzyciem kwartalow

In [546]:
def transform_quarter_to_number(string) -> int:
    if ' ' not in string:
        raise ValueError(f"Invalid format for quarter: {string}")
    quarter, year = string.split(' ')
    quarters = {'Q1': 0, 'Q2': 1, 'Q3': 2, 'Q4': 3}
    year = int(year) - 8
    return year * 4 + quarters[quarter]


def transform_number_to_quarter(number) -> str:
    year = (number // 4) + 8
    quarters = {0: 'Q1', 1: 'Q2', 2: 'Q3', 3: 'Q4'}
    quarter = quarters[number % 4]
    quarter_string = quarter + ' {:02d}'.format(year)
    return quarters[quarter] + ' ' + str(year)


In [547]:
user_count_df['Q'] = user_count_df['Q'].astype(str).map(transform_quarter_to_number)
user_count_test_df['Q'] = user_count_test_df['Q'].astype(str).map(transform_quarter_to_number)
print(user_count_df)
print(user_count_test_df)

     Q  User count
0    2         100
1    4         197
2    5         242
3    6         305
4    7         360
5    8         431
6    9         482
7   10         550
8   11         608
9   12         680
10  13         739
11  14         800
12  15         845
13  16         901
14  17         955
15  18        1007
16  19        1056
17  20        1110
18  21        1155
19  22        1189
20  23        1228
21  24        1276
22  25        1317
23  26        1350
24  27        1393
25  28        1441
26  29        1490
27  30        1545
28  31        1591
29  32        1654
30  33        1712
31  34        1788
32  35        1860
33  36        1936
34  37        2006
35  38        2072
36  39        2129
     Q  User count
0   40        2196
1   41        2234
2   42        2271
3   43        2320
4   44        2375
5   45        2414
6   46        2449
7   47        2498
8   48        2603
9   49        2701
10  50        2740
11  51        2797
12  52        2853
13  53      

## Trenowanie modelu szeregu czasowego 

In [548]:
R_train = user_count_df['Q'].to_numpy().reshape(-1, 1)
U_train = user_count_df['User count'].to_numpy().reshape(-1, 1)
R_test = user_count_test_df['Q'].to_numpy().reshape(-1, 1)
U_test = user_count_test_df['User count'].to_numpy().reshape(-1, 1)

linear_regression = LinearRegression()
linear_regression.fit(R_train, U_train)

# Ocena modelu na danych testowych
train_metrics = calculate_prediction_metrics(U_train, linear_regression.predict(R_train))
test_metrics = calculate_prediction_metrics(U_test, linear_regression.predict(R_test))

print_metrics(train_metrics, "Model szeregu czasowego (dane treningowe)")
print_metrics(test_metrics, "Model szeregu czasowego (dane testowe)")

print(linear_regression)

print(f'User count R^2: {linear_regression.score(R_train, U_train):}')
print(f"Model: U = {linear_regression.intercept_[0]:} + {linear_regression.coef_[0][0]:} * R")


Metryki oceny dla Model szeregu czasowego (dane treningowe):
------------------------------------------------------------
Metryka    Wartość         Jednostka       Ocena          
------------------------------------------------------------
MSE              1309.6367 (jednostka²)                   
RMSE               36.1889 (jednostka)                    
MAE                32.0882 (jednostka)                    
MAPE                4.2949 %               Bardzo dobry   
R2                  0.9959 (0-1)           Bardzo dobry   

Metryki oceny dla Model szeregu czasowego (dane testowe):
------------------------------------------------------------
Metryka    Wartość         Jednostka       Ocena          
------------------------------------------------------------
MSE              5608.8566 (jednostka²)                   
RMSE               74.8923 (jednostka)                    
MAE                63.4280 (jednostka)                    
MAPE                2.3554 %               Ba

## Errors for out model

In [549]:
errors = calculate_model_stats(linear_regression.predict(R_train), U_train, R_train)
# print(errors)

print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")

Errors for model:
Standard errror a0: 13.4365
Standard errror a1: 0.5704
Standard error e: 37.2085


## Wizualizacja modelu szeregu czasowego

In [550]:
# Funckja zmieniona w celu lepszej czytelnosci
def transform_number_to_quarter_custom(number) -> str:
    quarters = {0: 'Q1', 1: 'Q2', 2: 'Q3', 3: 'Q4'}
    year = 2008 + (number // 4)
    quarter = quarters[number % 4]
    return f'{quarter} {year}'


# Przygotowanie danych
train_predictions = linear_regression.predict(R_train)
test_predictions = linear_regression.predict(R_test)

# Obliczanie przedziału ufności
mse = mean_squared_error(U_train, train_predictions)
std_dev = np.sqrt(mse)
confidence_interval = 1.96 * std_dev  # 95% przedział ufności

# Tworzenie wykresu z większym rozmiarem i lepszymi proporcjami
plt.figure(figsize=(16, 9))


# Konfiguracja stylu
plt.style.use('seaborn-v0_8-deep')
colors = {
    'train_dots': '#1f77b4',
    'train_line': '#ff0000',
    'test_dots': '#2ca02c',
    'test_line': '#ff7f0e',
    'confidence': '#fff59d'
}

# Dane treningowe
plt.scatter(R_train, U_train, color=colors['train_dots'], alpha=0.7, s=60,
            label='Rzeczywista liczba użytkowników (dane uczące)',
            edgecolor='white', linewidth=1)
plt.plot(R_train, train_predictions, color=colors['train_line'], linewidth=2.5,
         label='Prognozowana liczba użytkowników (dane uczące)')

# Dane testowe
plt.scatter(R_test, U_test, color=colors['test_dots'], alpha=0.7, s=60,
            label='Rzeczywista liczba użytkowników (dane testowe)',
            edgecolor='white', linewidth=1)
plt.plot(R_test, test_predictions, color=colors['test_line'], linewidth=2.5,
         label='Prognozowana liczba użytkowników (dane testowe)')

# Przedział ufności z lepszą przezroczystością i kolorem
all_quarters = np.concatenate([R_train.flatten(), R_test.flatten()])
all_predictions = np.concatenate([train_predictions.flatten(), test_predictions.flatten()])
plt.fill_between(
    all_quarters,
    all_predictions - confidence_interval,
    all_predictions + confidence_interval,
    color=colors['confidence'],
    alpha=0.3,
    label='95% przedział ufności'
)

# Formatowanie osi i etykiet
plt.xlabel('Kwartał', fontsize=12, fontweight='bold')
plt.ylabel('Liczba użytkowników [mln]', fontsize=12, fontweight='bold')
plt.title('Prognoza liczby użytkowników Facebooka w latach 2008-2022',
          fontsize=16, pad=20, fontweight='bold')

# Generowanie etykiet osi X używając zdefiniowanych funkcji
x_ticks = range(0, 60)
x_labels = [transform_number_to_quarter_custom(q) for q in x_ticks]
plt.xticks(x_ticks, x_labels, rotation=45, ha='right')

# Dodanie siatki z lepszą widocznością
plt.grid(True, linestyle='--', alpha=0.3)

# Lepsze umiejscowienie legendy i dodanie ramki
legend = plt.legend(bbox_to_anchor=(1.15, 1), loc='upper left',
                    frameon=True, fancybox=True, shadow=True)
legend.get_frame().set_alpha(0.9)

# Dodanie informacji o R² w lepszym formacie
r2_score = linear_regression.score(R_train, U_train)
stats_text = f'R² (test) = {r2_score:.4f}\n'
stats_text += f'RMSE = {np.sqrt(mse):.2f} mln'

plt.text(0.02, 0.95, stats_text,
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='white',
                   edgecolor='gray',
                   alpha=0.8,
                   boxstyle='round,pad=0.5'),
         fontsize=10)

# Dodanie adnotacji dla ważnych punktów
max_users_idx = np.argmax(U_test)
plt.annotate(f'Maksimum: {U_test[max_users_idx][0]:.0f} mln',
             xy=(R_test[max_users_idx], U_test[max_users_idx]),
             xytext=(10, 10), textcoords='offset points',
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.7),
             arrowprops=dict(arrowstyle='->'))

# Dostosowanie układu i marginesów 
plt.tight_layout()

# Pokazanie wykresu
plt.show()

<IPython.core.display.Javascript object>

## Wykres user_count w zaleznosci od income

In [551]:
# Przygotowanie danych treningowych (2008-2017)
X_train = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)  # zmienna niezależna
y_train = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)      # zmienna zależna

# Przygotowanie danych testowych (2018-2022)
X_test = fb_stats_y_test_df['User count'].to_numpy().reshape(-1, 1)
y_test = fb_stats_y_test_df['Income'].to_numpy().reshape(-1, 1)

# Trenowanie modelu
linear_regression_income = LinearRegression()
linear_regression_income.fit(X_train, y_train)  # X_train jako pierwsza zmienna

LinearRegression()

In [552]:
# Statystyki modelu: wspolczynnik determinacji R^2
print(f'Income R^2: {linear_regression_income.score(X_train, y_train):}')

# Równanie modelu ma postać: I = wyraz wolny + współczynnik * U
# - 'intercept_' to wyraz wolny (przecięcie z osią y) linii regresji, reprezentujący przewidywaną wartość I, gdy U wynosi 0.
# - 'coef_' to nachylenie linii regresji, wskazujące, o ile zmienia się I przy jednostkowej zmianie U.
print(
    f'Model: I {linear_regression_income.intercept_[0]:} + {linear_regression_income.coef_[0][0]:} * U')  # Model rowna sie wspolczynnikowi nachylenia + wyrazowi wolnemu

Income R^2: 0.8297488143154316
Model: I -8936.98669619589 + 18.598913783523628 * U


## Błędy modelu

In [553]:
# Przeliczanie błędów modelu
errors = calculate_model_stats(linear_regression_income.predict(X_train), y_train, X_train)
# print(errors)

print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")

Errors for model:
Standard errror a0: 3804.0514
Standard errror a1: 2.9786
Standard error e: 5831.9193


In [554]:
# Przewidywania dla danych treningowych
train_predictions = linear_regression_income.predict(X_train)
print("Przewidywania dla danych treningowych:")
print(train_predictions)
print("\n" + "=" * 30 + "\n")  # Separator dla lepszej czytelności

# Przewidywania dla danych testowych
test_predictions = linear_regression_income.predict(X_test)
print("Przewidywania dla danych testowych:")
print(test_predictions)

Przewidywania dla danych treningowych:
[[-7077.09531784]
 [-2241.37773413]
 [ 2371.15288419]
 [ 6779.09545088]
 [10703.46625921]
 [13902.47942997]
 [16971.30020425]
 [20653.88513339]
 [25656.99294116]
 [30660.10074893]]


Przewidywania dla danych testowych:
[[34212.49328158]
 [37523.09993505]
 [43084.17515632]
 [45223.05024142]
 [46171.59484438]]


## Zmiana koncepcji testowanie modelu dla usunietych 3 pierwszych danych dla mniej problematycznej prostej

In [555]:
# Przygotowanie danych treningowych i testowych
# U_train_1 i P_train_1 zawierają wszystkie dane, podczas gdy U_train_2 i P_train_2 pomijają pierwsze trzy punkty
U_train_1 = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)
P_train_1 = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)
P_train_log_1 = np.log10(P_train_1)  # Logarytmowanie przychodów dla lepszej liniowości

# Narysuj wykresy dla obu zestawów danych
plt.figure(figsize=(10, 6))
plt.grid(True, linestyle='--', alpha=0.3)
plt.scatter(U_train_1, P_train_log_1, label='Przychód rzeczywisty')
plt.xlabel('Liczba użytkowników [mln]')
plt.ylabel('Przychód [mln]')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.grid(True, linestyle='--', alpha=0.3)
plt.scatter(U_train_1, P_train_1, label='Przychód rzeczywisty')
plt.xlabel('Liczba użytkowników [mln]')
plt.ylabel('Przychód [mln]')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Logarytmizacja danych oraz testowanie modelu liniowej regresji dla zadanych punktow

In [556]:
# Przygotowanie danych treningowych i testowych
U_train_1 = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)
P_train_1 = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)
P_train_log_1 = np.log10(P_train_1)  # Logarytmowanie przychodów dla lepszej liniowości

# Ile punktów chcemy usunąć
points_to_remove = 3

# Wydzielenie usuniętych punktów i pozostałych
U_removed = U_train_1[:points_to_remove]  # Pierwsze 3 punkty
P_removed_log = P_train_log_1[:points_to_remove]  # Logarytmy pierwszych 3 punktów
U_kept = U_train_1[points_to_remove:]  # Pozostałe punkty
P_kept_log = P_train_log_1[points_to_remove:]  # Logarytmy pozostałych punktów

# Tworzenie i trenowanie modeli regresji liniowej
l1 = LinearRegression()
l1.fit(U_train_1, P_train_log_1)  # Model uwzględniający wszystkie punkty

l2 = LinearRegression()
l2.fit(U_kept, P_kept_log)  # Model pomijający pierwsze trzy punkty

# Pobieranie współczynników i wyrazów wolnych z obu modeli
a11 = l1.coef_[0]
a01 = l1.intercept_
a12 = l2.coef_[0]
a02 = l2.intercept_

# Przewidywanie przychodów na podstawie obu modeli
P_pred_1 = a11 * U_train_1 + a01
P_pred_2 = a12 * U_train_1 + a02

# Wizualizacja danych i modeli
plt.clf()
plt.grid(True, linestyle='--', alpha=0.3)

# Punkty zachowane na zielono
plt.scatter(U_kept, P_kept_log, color='green', label='Punkty zachowane')
# Punkty usunięte na czerwono
plt.scatter(U_removed, P_removed_log, color='red', label='Punkty usunięte')

plt.plot(U_train_1, P_pred_1, label='Linia uwzględniająca wszystkie punkty')
plt.plot(U_train_1, P_pred_2, label='Linia bez 3 pierwszych punktów')

plt.xlabel('Liczba użytkowników [mln]')
plt.ylabel('Przychód [log10(mln)]')
plt.legend()
plt.show()

## Bledy modelu

In [557]:
# Przygotowanie danych do ostatecznego modelu
U_train = fb_stats_y_df['User count'].to_numpy()[3:].reshape(-1, 1)
P_train = fb_stats_y_df['Income'].to_numpy()[3:].reshape(-1, 1)
P_train_log = np.log10(P_train)

U_test = fb_stats_y_test_df['User count'].to_numpy().reshape(-1, 1)
P_test = fb_stats_y_test_df['Income'].to_numpy().reshape(-1, 1)
P_test_log = np.log10(P_test)

# Trenowanie ostatecznego modelu regresji liniowej
lin_reg_income = LinearRegression()
lin_reg_income.fit(U_train, P_train_log)

# Ocena modelu
print(f'Income R^2: {lin_reg_income.score(U_train, P_train_log)}')
errors = calculate_model_stats(lin_reg_income.predict(U_train), P_train_log, U_train)
print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")
    
# print(errors) # For the a1 bc -05e
P_error = errors['Standard error e']

# Ocena modelu na danych zlogarytmowanych
train_metrics = calculate_prediction_metrics(P_train_log, lin_reg_income.predict(U_train))
test_metrics = calculate_prediction_metrics(P_test_log, lin_reg_income.predict(U_test))

print_metrics_with_specific_units(train_metrics, "Model przychodów zlogarytmowanych (dane treningowe)", "log10(mln $)")
print_metrics_with_specific_units(test_metrics, "Model przychodów zlogarytmowanych (dane testowe)", "log10(mln $)")

Income R^2: 0.9880042958405737
Errors for model:
Standard errror a0: 0.0623
Standard errror a1: 0.0000
Standard error e: 0.0458

Metryki oceny dla Model przychodów zlogarytmowanych (dane treningowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------
MSE                 0.0015 (log10(mln $)²)                     
RMSE                0.0387 (log10(mln $))                      
MAE                 0.0323 (log10(mln $))                      
MAPE                0.7837 %                    Bardzo dobry   
R2                  0.9880 (0-1)                Bardzo dobry   

Metryki oceny dla Model przychodów zlogarytmowanych (dane testowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------

## Wizualizacja modelu

In [558]:
# Pobieranie współczynników i wyrazu wolnego z ostatecznego modelu
a1 = lin_reg_income.coef_[0]
a0 = lin_reg_income.intercept_

# Przewidywanie przychodów dla danych treningowych i testowych
P_train_pred = a1 * U_train + a0
P_test_pred = a1 * U_test + a0

# Wizualizacja wyników modelu
plt.figure(figsize=(10, 6))
# Dodanie siatki z lepszą widocznością
plt.grid(True, linestyle='--', alpha=0.3)
plt.fill_between(U_train.reshape(-1), 
                 (P_train_pred - P_error).reshape(-1), 
                 (P_train_pred + P_error).reshape(-1), 
                 color='yellow', 
                 label='Odchylenie standardowe prognozy (dane uczące)')
plt.fill_between(U_test.reshape(-1), 
                 (P_test_pred - P_error).reshape(-1), 
                 (P_test_pred + P_error).reshape(-1), 
                 color='yellow', 
                 alpha=0.5, 
                 label='Odchylenie standardowe prognozy (dane testowe)')
plt.scatter(U_train, P_train_log, label="Przychód prawdziwy (dane uczące)")
plt.plot(U_train, P_train_pred, label='Przychód przewidywany (dane uczące)', color='red')
plt.scatter(U_test, P_test_log, label="Przychód prawdziwy (dane testowe)", color='green')
plt.plot(U_test, P_test_pred, label='Przychód przewidywany (dane testowe)', color='orange')
plt.ylabel('Przychód [mln]')
plt.xlabel('Liczba użytkowników [mln]')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

## Wykres zależności Costs od Employment (Jak kształtują się koszty w zależności od zatrudnienia)

In [559]:
#  Wykres zależności Costs od Employment (Jak kształtują się koszty w zależności od zatrudnienia)
X_train = fb_stats_y_df['Employment'].to_numpy().reshape(-1, 1)
y_train = fb_stats_y_df['Costs'].to_numpy().reshape(-1, 1)
X_test = fb_stats_y_test_df['Employment'].to_numpy().reshape(-1, 1)
y_test = fb_stats_y_test_df['Costs'].to_numpy().reshape(-1, 1)

linear_regression_cost = LinearRegression()
linear_regression_cost.fit(X_train, y_train)

print(linear_regression_cost)
print(f'Cost R^2: {linear_regression_cost.score(X_train, y_train):}')
print(f"Model: P = {linear_regression_cost.intercept_[0]:} + {linear_regression_cost.coef_[0][0]:} * X")


LinearRegression()
Cost R^2: 0.9935274882125147
Model: P = -275.52690740247454 + 1.0319230656846947 * X


# Standardowe błędy:

In [560]:
errors = calculate_model_stats(linear_regression_cost.predict(X_train), y_train, X_train)
# print(errors)

print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")
    
train_metrics = calculate_prediction_metrics(y_train, linear_regression_cost.predict(X_train))
test_metrics = calculate_prediction_metrics(y_test, linear_regression_cost.predict(X_test))

print_metrics_with_specific_units(train_metrics, "Model kosztów w zależności od zatrudnienia (dane treningowe)", "mln $")
print_metrics_with_specific_units(test_metrics, "Model kosztów w zależności od zatrudnienia (dane testowe)", "mln $")

Errors for model:
Standard errror a0: 328.5797
Standard errror a1: 0.0294
Standard error e: 700.6804

Metryki oceny dla Model kosztów w zależności od zatrudnienia (dane treningowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------
MSE            392762.4715 (mln $²)                            
RMSE              626.7076 (mln $)                             
MAE               497.1730 (mln $)                             
MAPE               24.4464 %                    Umiarkowany    
R2                  0.9935 (0-1)                Bardzo dobry   

Metryki oceny dla Model kosztów w zależności od zatrudnienia (dane testowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------
MSE     

## Wizualizcja modelu Kosztów od Zatrudnienia:

In [561]:
# Przygotowanie danych
train_predictions = linear_regression_cost.predict(X_train)
test_predictions = linear_regression_cost.predict(X_test)

# Obliczanie przedziału ufności
mse = mean_squared_error(y_train, train_predictions)
std_dev = np.sqrt(mse)
confidence_interval = 1.96 * std_dev  # 95% przedział ufności

# Tworzenie wykresu z większym rozmiarem i lepszymi proporcjami
plt.figure(figsize=(16, 9))

# Konfiguracja stylu
plt.style.use('seaborn-v0_8-deep')
colors = {
    'train_dots': '#1f77b4',  # Stonowany niebieski dla punktów treningowych
    'train_line': '#ff0000',  # Ciepły czerwony dla linii treningowej
    'test_dots': '#2ca02c',  # Łagodny zielony dla punktów testowych
    'test_line': '#ff7f0e',  # Jasny fiolet dla linii testowej
    'confidence': '#fff59d'  # Ciepły żółty dla przedziału ufności
}

# Dane treningowe
plt.scatter(X_train, y_train, color=colors['train_dots'], alpha=0.7, s=60,
            label='Rzeczywiste koszty (dane uczące)',
            edgecolor='white', linewidth=1)
plt.plot(X_train, train_predictions, color=colors['train_line'], linewidth=2.5,
         label='Prognozowane koszty (dane uczące)')

# Dane testowe
plt.scatter(X_test, y_test, color=colors['test_dots'], alpha=0.7, s=60,
            label='Rzeczywiste koszty (dane testowe)',
            edgecolor='white', linewidth=1)
plt.plot(X_test, test_predictions, color=colors['test_line'], linewidth=2.5,
         label='Prognozowane koszty (dane testowe)')

# Przedział ufności z lepszą przezroczystością i kolorem
all_employment = np.concatenate([X_train.flatten(), X_test.flatten()])
all_predictions = np.concatenate([train_predictions.flatten(), test_predictions.flatten()])
plt.fill_between(
    all_employment,
    all_predictions - confidence_interval,
    all_predictions + confidence_interval,
    color=colors['confidence'],
    alpha=0.3,
    label='95% przedział ufności'
)

# Formatowanie osi i etykiet
plt.xlabel('Liczba zatrudnień', fontsize=12, fontweight='bold')
plt.ylabel('Koszty [mln]', fontsize=12, fontweight='bold')
plt.title('Prognoza kosztów Facebooka w zależności od liczby zatrudnień',
          fontsize=16, pad=20, fontweight='bold')

# Dodanie siatki z lepszą widocznością
plt.grid(True, linestyle='--', alpha=0.3)

# Lepsze umiejscowienie legendy i dodanie ramki
legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
                    frameon=True, fancybox=True, shadow=True)
legend.get_frame().set_alpha(0.9)

# Dodanie informacji o R² w lepszym formacie
r2_score = linear_regression_cost.score(X_train, y_train)
stats_text = f'R² (test) = {r2_score:.4f}\n'
stats_text += f'RMSE = {np.sqrt(mse):.2f} mln'

plt.text(0.02, 0.95, stats_text,
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='white',
                   edgecolor='gray',
                   alpha=0.8,
                   boxstyle='round,pad=0.5'),
         fontsize=10)

# Dodanie adnotacji dla ważnych punktów, jeśli potrzebne
max_cost_idx = np.argmax(y_test)
plt.annotate(f'Maksimum: {y_test[max_cost_idx][0]:.0f} mln',
             xy=(X_test[max_cost_idx], y_test[max_cost_idx]),
             xytext=(-50, 15), textcoords='offset points',
             bbox=dict(facecolor='white', edgecolor='gray', alpha=0.7),
             arrowprops=dict(arrowstyle='->'))

# Dostosowanie układu i marginesów 
plt.tight_layout()

# Pokazanie wykresu
plt.show()

<IPython.core.display.Javascript object>

## Drugi wykres - sprawdzenie

In [562]:
plt.figure(figsize=(10, 6))

# Przedział ufności dla danych uczących
plt.fill_between(X_train.reshape(-1),
                 (train_predictions - confidence_interval).reshape(-1),
                 (train_predictions + confidence_interval).reshape(-1),
                 color='yellow', label='Odchylenie standardowe prognozy')

# Przedział ufności dla danych testowych
plt.fill_between(X_test.reshape(-1),
                 (test_predictions - confidence_interval).reshape(-1),
                 (test_predictions + confidence_interval).reshape(-1),
                 color='yellow')

# Punkty rzeczywistych kosztów dla danych treningowych
plt.scatter(X_train, y_train, label="Koszty rzeczywiste (dane uczące)")

# Linia przewidywanych kosztów dla danych treningowych
plt.plot(X_train, train_predictions, label='Koszty przewidywane (dane uczące)', color='red')

# Punkty rzeczywistych kosztów dla danych testowych
plt.scatter(X_test, y_test, label="Koszty rzeczywiste (dane testowe)", color='green')

# Linia przewidywanych kosztów dla danych testowych
plt.plot(X_test, test_predictions, label='Koszty przewidywane (dane testowe)', color='orange')

# Etykiety osi
plt.ylabel('Koszty [mln]')
plt.xlabel('Liczba zatrudnień')

# Legenda
plt.legend()

# Wyświetlenie wykresu
plt.show()


<IPython.core.display.Javascript object>

# Model kosztów w zależności od Zatrudnienia i Przychodu

In [563]:
Z_train = fb_stats_y_df['Employment'].to_numpy().reshape(-1, 1)
P_train = fb_stats_y_df['Income'].to_numpy().reshape(-1, 1)
ZP_train = np.column_stack((Z_train, P_train))
C_train = fb_stats_y_df['Costs'].to_numpy().reshape(-1, 1)

Z_test = fb_stats_y_test_df['Employment'].to_numpy().reshape(-1, 1)
P_test = fb_stats_y_test_df['Income'].to_numpy().reshape(-1, 1)
ZP_test = np.column_stack((Z_test, P_test))
C_test = fb_stats_y_test_df['Costs'].to_numpy().reshape(-1, 1)


In [564]:
Z_train_log = np.log10(Z_train)
P_train_log = np.log10(P_train)
ZP_train_log = np.log10(ZP_train)
C_train_log = np.log10(C_train)

Z_test_log = np.log10(Z_test)
P_test_log = np.log10(P_test)
ZP_test_log = np.log10(ZP_test)
C_test_log = np.log10(C_test)

In [565]:
linear_reg_costs = LinearRegression()
linear_reg_costs.fit(ZP_train_log, C_train_log)

print(f'Cost R^2: {linear_reg_costs.score(ZP_train_log, C_train_log):}')

Cost R^2: 0.9902417049772478


In [566]:
# Generowanie prognoz dla danych treningowych
y_hat = linear_reg_costs.predict(ZP_train_log)

# Rzeczywiste wartości kosztów (y_true)
y_true = C_train_log

# Macierz X dla funkcji `calculate_model_stats` - zawiera dane treningowe ZP_train
# Ta funkcja sama doda kolumnę jedynek, więc ZP_train powinno mieć tylko zmienne objaśniające.
X = ZP_train_log

# Użycie funkcji calculate_model_stats
#stats = calculate_model_stats(y_hat, y_true, X)

# Wyświetlenie wyników
#print("Statystyki modelu:", stats)
print("Errors for model:")
for key, value in errors.items():
    print(f"{key}: {float(value):.4f}")
train_metrics = calculate_prediction_metrics(C_train_log, linear_reg_costs.predict(ZP_train_log))
test_metrics = calculate_prediction_metrics(C_test_log, linear_reg_costs.predict(ZP_test_log))

print_metrics_with_specific_units(train_metrics, "Model wielowymiarowy kosztów (dane treningowe)", "mln $")
print_metrics_with_specific_units(test_metrics, "Model wielowymiarowy kosztów (dane testowe)", "mln $")

Errors for model:
Standard errror a0: 328.5797
Standard errror a1: 0.0294
Standard error e: 700.6804

Metryki oceny dla Model wielowymiarowy kosztów (dane treningowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------
MSE                 0.0036 (mln $²)                            
RMSE                0.0601 (mln $)                             
MAE                 0.0509 (mln $)                             
MAPE                1.4551 %                    Bardzo dobry   
R2                  0.9902 (0-1)                Bardzo dobry   

Metryki oceny dla Model wielowymiarowy kosztów (dane testowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------
MSE                 0.0036 (mln $²) 

In [580]:
%matplotlib notebook

# Znacznie szersze zakresy dla surface plot -> paramtry: gestosc siatki(100), zakresy osi x i y
axis_x = np.linspace(-5000, 100000, 100)  # Zwiększony zakres dla liczby pracowników
axis_y = np.linspace(-5000, 150000, 100)  # Zwiększony zakres dla przychodów
X, Y = np.meshgrid(axis_x, axis_y)

# Przekształć siatkę punktów do formatu wymaganego przez model
grid = np.vstack([X.ravel(), Y.ravel()]).T
Z = linear_reg_costs.predict(grid).reshape(X.shape)

fig = plt.figure(facecolor='white', figsize=(10, 8))
ax = fig.add_subplot(projection='3d')
ax.set_facecolor('white')

# Wykreślenie danych treningowych i testowych
ax.scatter(Z_train_log, P_train_log, C_train_log, label='Koszty prawdziwe(dane treningowe)', color='blue')
ax.scatter(Z_test_log, P_test_log, C_test_log, label='Koszty prawdziwe(dane testowe)', color='green')

# Wykreślenie powierzchni predykcji
surf = ax.plot_surface(X, Y, Z, alpha=0.3, label='Predykcje kosztow', color='orange')
surf._edgecolors2d = surf._edgecolor3d
surf._facecolors2d = surf._facecolor3d

ax.legend()
ax.set_xlabel('Liczba pracownikow')
ax.set_ylabel('Przychody [mln]')
ax.set_zlabel('Koszty [mln]')
plt.show()

<IPython.core.display.Javascript object>

## Placeholder + 1

# Model kosztów w zależności od zatrudnienia i liczby użytkowników

In [568]:
E_train = fb_stats_y_df['Employment'].to_numpy().reshape(-1, 1)
Us_train = fb_stats_y_df['User count'].to_numpy().reshape(-1, 1)
EUs_train = np.column_stack((Z_train, P_train))
Cost_train = fb_stats_y_df['Costs'].to_numpy().reshape(-1, 1)

E_test = fb_stats_y_test_df['Employment'].to_numpy().reshape(-1, 1)
Us_test = fb_stats_y_test_df['User count'].to_numpy().reshape(-1, 1)
EUs_test = np.column_stack((Z_test, P_test))
Cost_test = fb_stats_y_test_df['Costs'].to_numpy().reshape(-1, 1)

In [569]:
linear_reg_costs2 = LinearRegression()
linear_reg_costs2.fit(EUs_train, Cost_train)

print(f'Cost R^2: {linear_reg_costs2.score(EUs_train, Cost_train):}')

Cost R^2: 0.9973388279128372


In [570]:
# Generowanie prognoz dla danych treningowych
y_hat = linear_reg_costs2.predict(EUs_train)

# Rzeczywiste wartości kosztów (y_true)
y_true = Cost_train

# Macierz X dla funkcji `calculate_model_stats` - zawiera dane treningowe ZP_train
# Ta funkcja sama doda kolumnę jedynek, więc ZP_train powinno mieć tylko zmienne objaśniające.
X = EUs_train

# Użycie funkcji calculate_model_stats
stats = calculate_model_stats(y_hat, y_true, X)

# Wyświetlenie wyników
print("Statystyki modelu:", stats)

train_metrics = calculate_prediction_metrics(C_train, linear_reg_costs.predict(ZP_train))
test_metrics = calculate_prediction_metrics(C_test, linear_reg_costs.predict(ZP_test))

print_metrics_with_specific_units(train_metrics, "Model wielowymiarowy kosztów (dane treningowe)", "mln $")
print_metrics_with_specific_units(test_metrics, "Model wielowymiarowy kosztów (dane testowe)", "mln $")

Statystyki modelu: {'Standard errror a0': np.float64(430.3231283942176), 'Standard errror a1': np.float64(0.3117458090409852), 'Standard errror a2': np.float64(0.18554975542477403), 'Standard error e': np.float64(480.3038046694041)}

Metryki oceny dla Model wielowymiarowy kosztów (dane treningowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka            Ocena          
----------------------------------------------------------------------
MSE          18359022.2726 (mln $²)                            
RMSE             4284.7430 (mln $)                             
MAE              2639.8677 (mln $)                             
MAPE               40.4584 %                    Słaby          
R2                  0.6975 (0-1)                Umiarkowany    

Metryki oceny dla Model wielowymiarowy kosztów (dane testowe):
----------------------------------------------------------------------
Metryka    Wartość         Jednostka   

In [571]:
%matplotlib notebook

# Znacznie szersze zakresy dla surface plot -> paramtry: gestosc siatki(100), zakresy osi x i y
axis_x = np.linspace(-5000, 100000, 100)  # Zwiększony zakres dla liczby pracowników
axis_y = np.linspace(-5000, 150000, 100)  # Zwiększony zakres dla przychodów
X, Y = np.meshgrid(axis_x, axis_y)

# Przekształć siatkę punktów do formatu wymaganego przez model
grid = np.vstack([X.ravel(), Y.ravel()]).T
Z = linear_reg_costs.predict(grid).reshape(X.shape)

fig = plt.figure(facecolor='white', figsize=(10, 8))
ax = fig.add_subplot(projection='3d')
ax.set_facecolor('white')

# Wykreślenie danych treningowych i testowych
ax.scatter(E_train, Us_train, Cost_train, label='Koszty prawdziwe(dane treningowe)', color='blue')
ax.scatter(E_test, Us_test, Cost_test, label='Koszty prawdziwe(dane testowe)', color='green')

# Wykreślenie powierzchni predykcji
surf = ax.plot_surface(X, Y, Z, alpha=0.3, label='Predykcje kosztow', color='orange')
surf._edgecolors2d = surf._edgecolor3d
surf._facecolors2d = surf._facecolor3d

ax.legend()
ax.set_xlabel('Liczba pracownikow')
ax.set_ylabel('Liczba użytkowników [mln]')
ax.set_zlabel('Koszty [mln]')
plt.show()

<IPython.core.display.Javascript object>